<a href="https://colab.research.google.com/github/DrMiracle/Colab-Projects/blob/main/probability%20matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

10 квітня 2021 року я хочу поїхати в Одесу відпочити. Мені треба вирішити, чи брати теплі речі та чи брати дощовик. Дощовик мені потрібен, якщо буде дощ, теплі речі - якщо буде холодно. Залежно від того, яка буде погода, мені буде жарко, холодно чи нормально. Беремо дані звідси(https://meteopost.com/weather/archive/) і, порівнюючи з погодою за роки 2012-2020, бачимо, що: ймовірність, що буде дощ $(\frac{4}{10})$, ймовірність, що буде холодно $(\frac{7}{10})$.

Модель $\check{M} = <\Omega, D, R, \sigma>$, де:

$D = \{d_1, d_2, d_3, d_4\}$, ($d_1$: не взяв нічого; $d_2$: взяв теплі речі; $d_3$: взяв дощовик; $d_4$: взяв теплі речі і дощовик).

$R = \{r_1, r_2, r_3\}$, ($r_1$: жарко, $r_2$: холодно, $r_3$: нормально).

$\Omega = \{ω_1 (P = (\frac{3}{10}) \times (\frac{6}{10})), ω_2 (P = (\frac{3}{10}) \times  (\frac{4}{10})), ω_3 (P = (\frac{7}{10}) \times (\frac{6}{10})), ω_4 (P = (\frac{7}{10}) \times (\frac{4}{10}))\}$ ($ω_1$: тепло, нема дощу; $ω_2$: тепло, є дощ; $ω_3$: холодно, нема дощу; $ω_4$: холодно, є дощ).

$\sigma: (\omega_1, d_1) \to r_3, (\omega_2, d_1) \to r_2, (\omega_3, d_1) \to r_2, (\omega_4, d_1) \to r_2, (\omega_1, d_2) \to r_1, (\omega_2, d_2) \to r_1, (\omega_3, d_2) \to r_3, (\omega_4, d_2) \to r_3, (\omega_1, d_3) \to r_3, (\omega_2, d_3) \to r_3, (\omega_3, d_3) \to r_2, (\omega_4, d_3) \to r_3, (\omega_1, d_4) \to r_1, (\omega_2, d_4) \to r_1, (\omega_3, d_4) \to r_3, (\omega_4, d_4) \to r_3$.

In [ ]:
import numpy as np

class Model:

  def __init__(self, sigma, D):
    assert sigma.ndim == 2
    assert len(sigma) == len(D)
    self.sigma = sigma
    self.D = D
    self.n_results = np.max(sigma)

  # операція представлення
  def representation(self, P):
    self.sigma = np.array([])
    self.D = np.array([])
    self.represent(P, np.zeros((len(P[0],))), 0)
    self.sigma = self.sigma.reshape(-1, len(P[0]))

  def represent(self, P, way, j):
    for i in range(len(P)):
      way[j] = i + 1
      if j + 1 == len(P[0]):
        prob = 1
        for k in range(len(P[0])):
          prob *= P[(way[k] - 1).astype(int), k]
        if prob > 0:
          self.sigma = np.append(self.sigma, way)
          self.D = np.append(self.D, prob)
      else:
        self.represent(P, way, j + 1)
  # проєкція
  def projection(self):
    p = np.zeros((self.n_results, len(self.sigma[0])))
    for j in range(len(self.sigma[0])):
      for i in range(len(self.sigma)):
        p[self.sigma[i, j] - 1, j] += self.D[i]
    return p

# Матриця наслідків
sigma = np.array([[3, 2, 2, 2], [1, 1, 3, 3], [3, 3, 2, 3], [1, 1, 3, 3]])
# ймовірності станів
D = np.array([(3/10) * (6/10),(3/10) * (4/10), (7/10) * (6/10), (7/10) * (4/10)], dtype=np.float32)
# Створюємо модель
M = Model(sigma, D)
# Виконуємо проєкцію і показуємо її
P = M.projection()
print("Проєкція:")
print(P)
# Сума ймовірностей для кожної дії
print("\nСума ймовірностей для кожної дії:")
print(np.sum(P, axis=0))
# Виконуємо операцію представлення і показуємо її
M.representation(P)
print("\nПредставлення:")
print(M.sigma)
# Виводимо кількість станів після операції представлення (як бачимо, вона збільшилась з 4 до 24)
print("\nКількість станів після операції представлення = {}".format(len(M.sigma)))
# Виводимо розподіл ймовірностей для станів
print("\nРозподіл ймовірностей:")
print(M.D)
# Та переконуємось, що їхня сума дорівнює одиниці
print("\nСума ймовірностей станів = {}".format(np.sum(M.D)))

Проєкція:
[[0.4        0.4        0.         0.        ]
 [0.         0.18000001 0.59999999 0.18000001]
 [0.59999999 0.41999999 0.4        0.81999999]]

Сума ймовірностей для кожної дії:
[0.99999999 0.99999999 0.99999999 0.99999999]

Представлення:
[[1. 1. 2. 2.]
 [1. 1. 2. 3.]
 [1. 1. 3. 2.]
 [1. 1. 3. 3.]
 [1. 2. 2. 2.]
 [1. 2. 2. 3.]
 [1. 2. 3. 2.]
 [1. 2. 3. 3.]
 [1. 3. 2. 2.]
 [1. 3. 2. 3.]
 [1. 3. 3. 2.]
 [1. 3. 3. 3.]
 [3. 1. 2. 2.]
 [3. 1. 2. 3.]
 [3. 1. 3. 2.]
 [3. 1. 3. 3.]
 [3. 2. 2. 2.]
 [3. 2. 2. 3.]
 [3. 2. 3. 2.]
 [3. 2. 3. 3.]
 [3. 3. 2. 2.]
 [3. 3. 2. 3.]
 [3. 3. 3. 2.]
 [3. 3. 3. 3.]]

Кількість станів після операції представлення = 24

Розподіл ймовірностей:
[0.01728    0.07872    0.01152    0.05248    0.007776   0.035424
 0.005184   0.023616   0.018144   0.08265599 0.012096   0.055104
 0.02592    0.11808    0.01728    0.07872    0.011664   0.053136
 0.007776   0.035424   0.027216   0.12398399 0.018144   0.08265599]

Сума ймовірностей станів = 0.999999970197678
